# Machine learning Algorithms I
## 5 Decision Trees & Random Forests

In this notebook, we will implement a random forest in Python. We'll start with a single decision tree and a simple problem, and then work our way to a random forest. Once we understand how a single decision tree works, we can transfer this knowledge to an entire forest of trees.

> really nice introduction: https://www.youtube.com/watch?v=J4Wdy0Wc_xQ&t=528s

#### RF Tutorial Aims:
* Random Forest classification with scikit-learn
* How random forests work
* How to use them for regression
* How to evaluate their performance


## 5.1. Decision Trees

### 5.1.2. What is a decision tree


<div>
<img src="5_RF_figures/Decisiontree.png" width="400"/>
</div>

A simple linear classifier will not be able to draw a boundary that separates the classes. The single decision tree will be able to completely separate the points because it essentially draws many repeated linear boundaries between points. A decision tree is a non-parametric model because the number of parameters grows with the size of the data.

* DTs are easy to built, use and interpret **BUT** they are limited!
* Their main disadvantage: INACCURAY! --> are not flexible with new samples

# Application example: Random Forest Regression Analysis of Stable Water Isotope Variability in Southern Africa’s Precipitation

**Geppert, Marielle; Hartmann, Kai; Kirchner, Ingo; Pfahl, Stephan; Struck, Ulrich; Riedel, Frank (2022). Precipitation Over Southern Africa: Moisture Sources and Isotopic Composition. JGR: Atmospheres, https://doi.org/10.1029/2022JD037005**

---

## Context of the Study
Southern Africa is characterized by arid and semi-arid landscapes and is particularly susceptible to extreme weather conditions. Intriguingly, over the last 100,000 years, extensive lakes have periodically formed in the central Kalahari desert, raising questions about historical changes in atmospheric circulation and precipitation patterns.

Geppert et al. conducted a study about the annual precipitation distributions throughout Southern Africa. They focused on the analysis of stable water isotope compositions, moisture transport pathways, and moisture sources.

Stable isotopes of hydrogen and oxygen (such as <sup>2</sup>H and <sup>18</sup>O) in water molecules vary slightly based on their source and the environmental conditions they have been through. By analyzing these isotopes, it is possible to trace the origins of water sources and to understand the pathways of moisture transport.


Changes in stable isotope ratios in precipitation can reveal shifts in atmospheric circulation patterns and climate. For instance, when water evaporates from the ocean, water molecules containing heavier isotopes of oxygen (<sup>18</sup>O) and hydrogen (<sup>2</sup>H or deuterium) are more likely to remain in the ocean. This results in a higher concentration of these heavier isotopes in the ocean, which is reflected in the δ<sup>2</sup>H and δ<sup>18</sup>O ratios of the ocean water. As the evaporated water forms precipitation and moves inland, separation continues and and further changes isotopic ratios occur. Analyzing these isotopic variations helps to reconstruct past precipitation regimes and thus provides insights into historical patterns of atmospheric circulation.

---

## Data Collection
The study involved collecting water samples of precipitation and different surface waters in southern Africa between 2016 and 2021. The map shows the sampling locations and the sample type, which is: ocean, spring, lake, precipitation and river. Furthermore, the data of eight Global Network for Isotopes in Precipitation (GNIP) stations has been used.

![Map](./bild1.png)

Sample locations

The data set and further information about the sampling process can be found
[here](https://doi.pangaea.de/10.1594/PANGAEA.944811).


Let us take a closer look at the data:

In [1]:
# Importieren der pandas- und requests-Bibliothek sowie des StringIO-Moduls
import pandas as pd
import requests
from io import StringIO

# Daten über URL einlesen
url = "https://doi.pangaea.de/10.1594/PANGAEA.944811?format=textfile"

response = requests.get(url)
IsoW06 = pd.read_csv(StringIO(response.text), sep = '\t', skiprows = 267, header = 1, encoding = "UTF-8", 
                     engine = 'python', on_bad_lines = 'skip')

In [2]:
# Anzeigen lassen der ersten 6 Dateneinträge
print(IsoW06.head(6))

          Event Sample ID  Latitude  Longitude   Date/Time Samp type  \
0  WaterSA_SLW1      SLW1 -33.88917   18.96917  2016-08-29     River   
1  WaterSA_SLW2      SLW2 -33.87800   19.03517  2016-08-29     River   
2  WaterSA_SLW3      SLW3 -33.93667   19.17000  2016-08-29     River   
3  WaterSA_SLW4      SLW4 -33.69350   19.32483  2016-08-29     River   
4  WaterSA_SLW5      SLW5 -33.54333   19.20733  2016-08-29     River   
5  WaterSA_SLW6      SLW6 -33.33367   19.87767  2016-08-30      Lake   

                                      Sample comment  δ18O H2O [‰ SMOW]  \
0                                     River at Pniel              -3.54   
1  River Berg; abundant with insect larvae; dam u...              -3.33   
2                         Minor waterfall; iron rich              -4.44   
3                 River; abundant with insect larvae              -4.28   
4                                         River Bree              -4.09   
5  Reservoir lake; under almost natural condi

The data set contains 188 samples and the following 11 variables: 
Event, Sample ID, Latitude, Longitude, Date/Time, Samp type, Sample comment, δ18O H2O [‰ SMOW], δD H2O [‰ SMOW], δ18O H2O std dev [±], δD H2O std dev [±]. The isotope ratios are expressed in the conventional delta notation (δ18O, δ2H) in per mil (‰) relative to VSMOW ([Vienna Standard Mean Ocean Water](https://en.wikipedia.org/wiki/Vienna_Standard_Mean_Ocean_Water).

## The Random Forest Algorithm in the Study

The Random Forest (RF) algorithm is applied to assess the relative importance of various meteorological variables on the stable isotope data. Therefore, the study uses the cforest() function from the R package party. One of the main advantages of the party package is its ability to handle both categorical and continuous predictor variables.


In the following, the assessment will be showcased using the (=δ18O
) isotope data.

In [3]:
# Daten über CSV-Datei einlesen
TrajIsoLC = pd.read_csv('./Geppert2019.csv',  header = 0, sep = ',')

FileNotFoundError: [Errno 2] No such file or directory: './Geppert2019.csv'

We will rename the variables mw18O to O18(=δ<sup>18</sup>O) for clarity and variables mwdD (=δ<sup>2</sup>H) and d.Excess are removed to focus on the O18 isotopes. Furthermore, we remove the country of sample origin.

In [ ]:
# zwei Variablen umbenennen und mehrere Variablen entfernen 
TrajIsoLC.rename(columns = {'mw18O': 'O18', 'Monat': 'month'}, inplace = True)  
TrajIsoLC = TrajIsoLC.drop(columns = ["mwdD", "d.Excess", "Land.Ocean", "Africa", "Oceans", "ISO"])

Then the data is subset, including only data where the explanatory fraction is greater than 0.6:

In [ ]:
# Daten mit einer explanatory fraction größer als 0,6 extrahieren (Wert zeigt, wie gut die Abhängigkeit der Variablen 
# erklärt werden kann - 0 erklärt keine Variation - 1 erklärt jede Variation)
IsoW06 = TrajIsoLC[TrajIsoLC["expl.frac"] > 0.6]

### Initial Model

We start with a performance test on our data for the RF algorithm by creating an initial RF model using the randomForest() function. It predicts δ18O
 based on all other variables in IsoW06 with 2000 trees.

In [ ]:
# Importieren von Modulen/Bibliotheken
from sklearn.preprocessing import OrdinalEncoder

# NaN-Werte entfernen und den Datentyp von vier Variablen von kategorisch in numerisch umzuwandeln (Voraussetzung 
# für das Machine Learning)
IsoW06 = IsoW06.dropna()
ord_enc = OrdinalEncoder()
IsoW06["Unnamed: 0"] = ord_enc.fit_transform(IsoW06[["Unnamed: 0"]])
IsoW06["Type_main"] = ord_enc.fit_transform(IsoW06[["Type_main"]])
IsoW06["type"] = ord_enc.fit_transform(IsoW06[["type"]])
IsoW06["RFZraster"] = ord_enc.fit_transform(IsoW06[["RFZraster"]])

# Eingangs- (X, alle Variablen außer O18) und Zieldaten (y, Variable O18) definieren
X = IsoW06.drop('O18', axis = 1)
y = IsoW06['O18']

In [ ]:
# Importieren von Modulen/Bibliotheken
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Initialwert und Anzahl der Bäume für den Random Forest festlegen
seed = 196
ntrees = 2000

# RandomForest erstellen, X und y definieren und Modell trainieren
model = RandomForestRegressor(n_estimators = ntrees, random_state = seed, max_features = 1.0,
                               min_samples_split = 2, min_samples_leaf = 1, max_depth = None)
model.fit(X, y)

In [ ]:
# Mittelwert der quadrierten Residuen und erklärte Varianz ausgeben lassen
print(f"Mean of squared residuals: {model.score(X, y)}")
print(f"% Var explained: {model.score(X, y) * 100}")

In each split of a tree, 18 variables were randomly chosen to determine the best split. About <font color='royalblue'>87.67% </font> of the variability in the δ18O values can be explained by the model.

By visualizing the model performance, we can observe how error decreases with the number of trees.

In [ ]:
# DAUERT 2-3 Minuten!
# Liste und Schleife für die Durchführung des Models mit unterschiedlichen Anzahlen an Bäumen erstellen
ntrees_range = list(range(1, 3001, 200))
errors = []

for ntrees in ntrees_range:
    model = RandomForestRegressor(n_estimators = ntrees, random_state = seed)
    model.fit(X, y)
    errors.append(model.score(X, y))

In [ ]:
# Importieren von Modulen/Bibliotheken
import matplotlib.pyplot as plt

# Plotten der error-Werte in Abhängigkeit der Anzahl der Bäume des Random Forests, um optimalen ntree-Wert festzulegen
plt.plot(ntrees_range, errors, marker = 'o')
plt.xlabel('trees')
plt.ylabel('Error')
plt.title('model')
plt.show()

In [ ]:
# kleineren Ausschnitt plotten, um Anzahl der Bäume ideal festzulegen
plt.plot(ntrees_range, errors, marker = 'o')
plt.xlabel('trees')
plt.ylabel('Error')
plt.title('model')
plt.xlim(60, max(ntrees_range)+50)
plt.ylim(0.85, max(errors)+0.01)
plt.show()

As shown in the figure, the mean squared residuals stabilize around <font color='royalblue'>1000 trees</font>. Thus, in the following step we use <font color='royalblue'>´ntree=1000´</font> and optimize the tree depth. 

### Hyper-parameter Tuning

Now, we will tune the parameter that determines the number of variables that are randomly sampled as candidates at each split (mtry). The numbers of trees and variables are crucial for the model performance. The authors applied a range of mtry from 1 to 52 while ntree is fixed at 10000 after [Behnamian et al. 2017](https://doi.org/10.1109/LGRS.2017.2745049) (a large number of trees ensures stable variable importance). This grid represents different combinations of hyper-parameters to be tested. In our example we used the above identified threshold of <font color='royalblue'>ntrees=1000</font> and mtry=2:54. 

In [ ]:
# Importieren von Modulen/Bibliotheken
import numpy as np

# Initialwert und Anzahl der Bäume für den Random Forest festlegen und Zufallszahlengenerator laufen lassen
seed = 196
ntrees = 1000
np.random.seed(seed)

# Hyperparameteroptimierung für das Modell durchführen und Identifikation der besten Modelle (bzgl. OOB-RMSE)
mtry_range = list(range(1, 54, 2))
hyper_grid = {'mtry': mtry_range, 'ntree': [ntrees] * len(mtry_range), 'OOB_RMSE': [0] * len(mtry_range)}

for i, params in enumerate(mtry_range):
    model = RandomForestRegressor(n_estimators = ntrees, max_features = params, oob_score = True, random_state = seed)
    model.fit(X, y)
    hyper_grid['OOB_RMSE'][i] = model.oob_score_
    
hyper_grid_df = pd.DataFrame(hyper_grid)
best_models = hyper_grid_df.sort_values(by = 'OOB_RMSE').head(10)
print(best_models)   

For each parameter combination, the [Out-of-Bag](https://en.wikipedia.org/wiki/Out-of-bag_error) (OOB) error is calculated. The best model with <font color='royalblue'>1000 trees</font> has an <font color='royalblue'>mtry value of 51 (OOB_RMSE: 0.102699).</font> 

### Final Model

Now, the final RF model with optimized parameters can be created. The varimp() function is used to calculate the importance of each variable in the model. This tells us which variables are most predictive of O18.

In [ ]:
# Importieren von Modulen/Bibliotheken
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

# Initialwert und Anzahl der Bäume für den Random Forest festlegen und Zufallszahlengenerator laufen lassen
seed = 196
ntrees = 1000
np.random.seed(seed)

In [ ]:
# Optimalen mtry-Wert finden (Anzahl der zu betrachtenden, zufällig ausgewählten Features für jeden Baum)
min_idx = np.argmin(hyper_grid['OOB_RMSE'])
mtry = hyper_grid['mtry'][min_idx]

In [ ]:
# Finales RandomForest erstellen und trainieren
model = RandomForestRegressor(n_estimators = ntrees, max_features = mtry, random_state = seed)
model.fit(X, y)

In [ ]:
# Statistiken über das Modell ausgeben lassen und die 10 wichtigsten Variablen extrahieren
stats = model.get_params()
feature_importance = model.feature_importances_
features = X.columns
var_imp_df = pd.DataFrame({'Feature': features, 'Importance': feature_importance})
var_imp_df = var_imp_df.sort_values(by = 'Importance', ascending = False).head(54)
var_imp_10 = var_imp_df.head(10)

In [ ]:
# RMSE (Root Mean Square Error) und R² (R-squared) für Bewertung des Modells extrahieren
y_pred = model.predict(X)
rmse = np.sqrt(mean_squared_error(y, y_pred))
r2 = r2_score(y, y_pred)

In [ ]:
# Plotten der wichtigsten Variablen
plt.figure(figsize = (8, 3))
sns.barplot(x = 'Importance', y = 'Feature', data = var_imp_10, color = 'gray')
plt.title("δ¹⁸O")
plt.xlabel("Importance score")
plt.ylabel("Feature")
plt.text(0, -1.5, f"ntree = {ntrees}, mtry = {mtry}\nRMSE = {rmse:.2f}, R² = {r2:.2f}", 
         fontsize = 10, ha = 'left', va = 'center')
plt.show()

The variable abbreviations shown in the plot have the following meanings:
- <span style="color:royalblue"><b>wmSnk_lon</b>: longitude at sampling site</span>
- <span style="color:royalblue"><b>wmSrc_lat</b>: latitude</span>
- <span style="color:royalblue"><b>wmSrc_nocfblh</b>: boundary layer height</span>
- <span style="color:royalblue"><b>wmSnk_lat</b>: latitude at sampling site</span>
- <span style="color:royalblue"><b>wmSrc_sd.nocfblh</b>: boundary layer height</span>
- <span style="color:royalblue"><b>wmSrc_sd.PS</b>: surface pressure difference</span>
- <span style="color:royalblue"><b>month</b>: month of sampling</span>
- <span style="color:royalblue"><b>wmSnk_nocftp</b>: total precipitation at sampling site</span>
- <span style="color:royalblue"><b>Type_main</b>: main water type</span>
- <span style="color:royalblue"><b>wmSrc_sd.lon</b>: longitude</span>

For predicting the δ18O ratio, <font color='royalblue'>the longitude at sampling site seems to be the most important variable, among latitude at target location, boundary layer height and latitude at sampling site.</font>

---

#### Citation
The E-Learning project SOGA-R was developed at the Department of Earth Sciences by Kai Hartmann, Joachim Krois and Annette Rudolph. You can reach us via mail by [soga[at]zedat.fu-berlin.de](soga@zedat.fu-berlin.de).

<img src="./bild2.png" alt="CC-BY-SA" style="float: left; margin-right: 10px;" /> &nbsp;

You may use this project freely under the [Creative Commons Attribution-ShareAlike 4.0 International License](http://creativecommons.org/licenses/by-sa/4.0/). &nbsp;

Please cite as follow: *Hartmann, K., Krois, J., Rudolph, A. (2023): Statistics and Geodata Analysis using R ([SOGA-R](https://www.geo.fu-berlin.de/soga-r)). Department of Earth Sciences, Freie Universitaet Berlin.*
